In [29]:
import pandas as pd

df = pd.read_csv(r'C:\Users\clint\Desktop\RER\Code\18.csv')
df

,Sending Country,Receiving Country,Year,Value,Unit,Source,Region
0,Algeria,Senegal,2021,0.183414825,USD millions,BCEAO,Africa
1,Australia,Ethiopia,2020,13.59617511,USD millions,National Bank of Ethiopia,Africa
2,Australia,Kenya,2024,"184,497.099695719",USD millions,Central Bank of Kenya,Africa
3,Australia,Uganda,2022,22,USD millions,Bank of Uganda,Africa
4,Austria,Kenya,2024,"13,169.065145833",USD millions,Central Bank of Kenya,Africa
...,...,...,...,...,...,...,...
3975,Suriname,United States,2019,5.022,USD millions,Roland Kpodar (IMF),North America
3976,Suriname,United States,2020,3.275,USD millions,Roland Kpodar (IMF),North America
3977,Suriname,Vietnam,2018,1.401,USD millions,Roland Kpodar (IMF),Asia
3978,Suriname,Vietnam,2019,1.453,USD millions,Roland Kpodar (IMF),Asia


In [30]:
# Install required packages for GDP data retrieval
import requests
import json
import numpy as np

# Function to get GDP data from World Bank API
def get_gdp_data(country_code, year):
    """
    Get GDP data for a specific country and year from World Bank API
    """
    url = f"https://api.worldbank.org/v2/country/{country_code}/indicator/NY.GDP.MKTP.CD?date={year}&format=json"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if len(data) > 1 and data[1]:
                return data[1][0]['value']
    except:
        pass
    return None

# Create a mapping of country names to World Bank country codes
country_codes = {
    'Algeria': 'DZA', 'Australia': 'AUS', 'Austria': 'AUT', 'Bahrain': 'BHR',
    'Belgium': 'BEL', 'Benin': 'BEN', 'Brazil': 'BRA', 'Bulgaria': 'BGR',
    'Burkina Faso': 'BFA', 'Cambodia': 'KHM', 'Canada': 'CAN', 'China': 'CHN',
    'Croatia': 'HRV', 'Czech Republic': 'CZE', 'Denmark': 'DNK', 'Egypt': 'EGY',
    'Estonia': 'EST', 'Ethiopia': 'ETH', 'Finland': 'FIN', 'France': 'FRA',
    'Germany': 'DEU', 'Ghana': 'GHA', 'Greece': 'GRC', 'India': 'IND',
    'Indonesia': 'IDN', 'Iran': 'IRN', 'Ireland': 'IRL', 'Israel': 'ISR',
    'Italy': 'ITA', 'Japan': 'JPN', 'Jordan': 'JOR', 'Kenya': 'KEN',
    'Kuwait': 'KWT', 'Lebanon': 'LBN', 'Luxembourg': 'LUX', 'Malaysia': 'MYS',
    'Mali': 'MLI', 'Morocco': 'MAR', 'Netherlands': 'NLD', 'Niger': 'NER',
    'Nigeria': 'NGA', 'Norway': 'NOR', 'Pakistan': 'PAK', 'Philippines': 'PHL',
    'Poland': 'POL', 'Qatar': 'QAT', 'Russia': 'RUS', 'Rwanda': 'RWA',
    'Saudi Arabia': 'SAU', 'Senegal': 'SEN', 'South Africa': 'ZAF',
    'South Korea': 'KOR', 'Spain': 'ESP', 'Sudan': 'SDN', 'Sweden': 'SWE',
    'Switzerland': 'CHE', 'Thailand': 'THA', 'Turkey': 'TUR', 'Uganda': 'UGA',
    'Ukraine': 'UKR', 'United Arab Emirates': 'ARE', 'United Kingdom': 'GBR',
    'United States': 'USA', 'Vietnam': 'VNM', 'Suriname': 'SUR'
}

# Get unique countries and years from the dataset
unique_countries = df['Sending Country'].unique()
unique_years = df['Year'].unique()

print(f"Found {len(unique_countries)} unique sending countries")
print(f"Years range from {min(unique_years)} to {max(unique_years)}")
print("Sample countries:", list(unique_countries)[:10])

Found 257 unique sending countries
Years range from 2018 to 2024
Sample countries: ['Algeria', 'Australia', 'Austria', 'Bahamas', 'Bahrain', 'Belgium', 'Benin', 'Brazil', 'Burkina Faso', 'Cameroon']


In [32]:
# Updated function to get inflation-adjusted GDP data from World Bank API
def get_real_gdp_data(country_code, year):
    """
    Get real GDP (constant 2015 US$) data for a specific country and year from World Bank API
    This is already inflation-adjusted to 2015 US dollars
    """
    url = f"https://api.worldbank.org/v2/country/{country_code}/indicator/NY.GDP.MKTP.KD?date={year}&format=json"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if len(data) > 1 and data[1] and len(data[1]) > 0:
                return data[1][0]['value']
    except Exception as e:
        print(f"Error fetching GDP data for {country_code}, {year}: {e}")
        pass
    return None

# Create a function to add the inflation-adjusted GDP column
def add_real_gdp_column(df):
    """
    Add a column with inflation-adjusted GDP (constant 2015 US$) for each sending country and year
    """
    real_gdp_values = []
    
    print("Fetching inflation-adjusted GDP data...")
    
    for idx, row in df.iterrows():
        country = row['Sending Country']
        year = row['Year']
        
        country_code = country_codes.get(country)
        if country_code:
            real_gdp = get_real_gdp_data(country_code, year)
            real_gdp_values.append(real_gdp)
            
            if idx % 500 == 0:  # Progress indicator every 500 rows
                print(f"Processed {idx+1}/{len(df)} rows...")
        else:
            real_gdp_values.append(None)
            if idx % 500 == 0:
                print(f"Country code not found for {country}")
    
    df['Sending_Country_Real_GDP'] = real_gdp_values
    return df

print("Functions defined. Ready to add inflation-adjusted GDP column.")

Functions defined. Ready to add inflation-adjusted GDP column.


In [34]:
# Test with a small sample first
print("Testing with a small sample of 10 rows...")

# Create a small test dataset
test_df = df.head(10).copy()
print("Test dataset:")
print(test_df[['Sending Country', 'Year']])

print("\nFetching GDP data for test sample...")
test_df_with_gdp = add_real_gdp_column(test_df)

print("\nResults:")
print(test_df_with_gdp[['Sending Country', 'Year', 'Sending_Country_Real_GDP']])

# Check which ones worked
successful = test_df_with_gdp['Sending_Country_Real_GDP'].notna().sum()
print(f"\nTest results: {successful}/10 rows successfully fetched GDP data")

Testing with a small sample of 10 rows...
Test dataset:
  Sending Country  Year
0         Algeria  2021
1       Australia  2020
2       Australia  2024
3       Australia  2022
4         Austria  2024
5         Bahamas  2024
6         Bahrain  2024
7         Bahrain  2020
8         Belgium  2024
9         Belgium  2020

Fetching GDP data for test sample...
Fetching inflation-adjusted GDP data...
Processed 1/10 rows...

Results:
  Sending Country  Year  Sending_Country_Real_GDP
0         Algeria  2021              1.994889e+11
1       Australia  2020              1.491063e+12
2       Australia  2024              1.665258e+12
3       Australia  2022              1.587133e+12
4         Austria  2024              4.181904e+11
5         Bahamas  2024                       NaN
6         Bahrain  2024              4.117271e+10
7         Bahrain  2020              3.472477e+10
8         Belgium  2024              5.325307e+11
9         Belgium  2020              4.704176e+11

Test results: 9/10

In [35]:
# Display the test results in a more readable format
print("Test Results Summary:")
print("="*50)

for idx, row in test_df_with_gdp.iterrows():
    country = row['Sending Country']
    year = row['Year']
    gdp = row['Sending_Country_Real_GDP']
    
    if pd.notna(gdp):
        gdp_trillions = gdp / 1e12
        print(f"{country} ({year}): ${gdp_trillions:.2f} trillion (constant 2015 USD)")
    else:
        print(f"{country} ({year}): GDP data not available")

print("\nThe new column 'Sending_Country_Real_GDP' contains:")
print("- Real GDP in constant 2015 US dollars (inflation-adjusted)")
print("- Values are already adjusted for inflation to the 2015 base year")
print("- This eliminates the need for manual inflation calculations")

print(f"\nSuccess rate in test: {9/10*100}% - This suggests good data availability")
print("\nOptions for full dataset:")
print("1. Process all 3,980 rows (will take ~20-30 minutes)")
print("2. Process in smaller batches")
print("3. Use a cached/pre-downloaded GDP dataset for faster processing")

Test Results Summary:
Algeria (2021): $0.20 trillion (constant 2015 USD)
Australia (2020): $1.49 trillion (constant 2015 USD)
Australia (2024): $1.67 trillion (constant 2015 USD)
Australia (2022): $1.59 trillion (constant 2015 USD)
Austria (2024): $0.42 trillion (constant 2015 USD)
Bahamas (2024): GDP data not available
Bahrain (2024): $0.04 trillion (constant 2015 USD)
Bahrain (2020): $0.03 trillion (constant 2015 USD)
Belgium (2024): $0.53 trillion (constant 2015 USD)
Belgium (2020): $0.47 trillion (constant 2015 USD)

The new column 'Sending_Country_Real_GDP' contains:
- Real GDP in constant 2015 US dollars (inflation-adjusted)
- Values are already adjusted for inflation to the 2015 base year
- This eliminates the need for manual inflation calculations

Success rate in test: 90.0% - This suggests good data availability

Options for full dataset:
1. Process all 3,980 rows (will take ~20-30 minutes)
2. Process in smaller batches
3. Use a cached/pre-downloaded GDP dataset for faster pr

In [36]:
# Let's investigate why Bahamas 2024 GDP data is missing
print("Investigating Bahamas GDP data...")

# Check if Bahamas is in our country codes mapping
print(f"Bahamas in country_codes mapping: {'Bahamas' in country_codes}")
if 'Bahamas' in country_codes:
    print(f"Bahamas country code: {country_codes['Bahamas']}")

# Let's try to get Bahamas GDP data for different years to see what's available
bahamas_code = 'BHS'  # Standard ISO code for Bahamas
years_to_test = [2020, 2021, 2022, 2023, 2024]

print(f"\nTesting Bahamas GDP data availability:")
for year in years_to_test:
    # Try the real GDP indicator
    url = f"https://api.worldbank.org/v2/country/{bahamas_code}/indicator/NY.GDP.MKTP.KD?date={year}&format=json"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if len(data) > 1 and data[1] and len(data[1]) > 0 and data[1][0]['value'] is not None:
                gdp_value = data[1][0]['value']
                print(f"  {year}: ${gdp_value/1e9:.2f} billion (constant 2015 USD)")
            else:
                print(f"  {year}: No data available")
        else:
            print(f"  {year}: API error (status code: {response.status_code})")
    except Exception as e:
        print(f"  {year}: Error - {e}")

# Let's also check what the actual API response looks like for 2024
print(f"\nDetailed API response for Bahamas 2024:")
url_2024 = f"https://api.worldbank.org/v2/country/{bahamas_code}/indicator/NY.GDP.MKTP.KD?date=2024&format=json"
try:
    response = requests.get(url_2024)
    if response.status_code == 200:
        data = response.json()
        print(f"Response structure: {len(data)} elements")
        if len(data) > 1:
            print(f"Data element: {data[1]}")
    else:
        print(f"API returned status code: {response.status_code}")
except Exception as e:
    print(f"Error: {e}")

Investigating Bahamas GDP data...
Bahamas in country_codes mapping: False

Testing Bahamas GDP data availability:
  2020: $9.81 billion (constant 2015 USD)
  2021: $11.54 billion (constant 2015 USD)
  2022: $12.80 billion (constant 2015 USD)
  2023: $13.19 billion (constant 2015 USD)
  2024: $13.63 billion (constant 2015 USD)

Detailed API response for Bahamas 2024:
Response structure: 2 elements
Data element: [{'indicator': {'id': 'NY.GDP.MKTP.KD', 'value': 'GDP (constant 2015 US$)'}, 'country': {'id': 'BS', 'value': 'Bahamas, The'}, 'countryiso3code': 'BHS', 'date': '2024', 'value': 13630935977.0689, 'unit': '', 'obs_status': '', 'decimal': 0}]


In [37]:
# Fix the country codes mapping - add missing countries
print("Updating country codes mapping...")

# Add Bahamas and other potentially missing countries
additional_countries = {
    'Bahamas': 'BHS',
    'Bahamas, The': 'BHS', 
    'The Bahamas': 'BHS',
    # Add a few more that might be missing
    'Congo, Rep.': 'COG',
    'Congo, Dem. Rep.': 'COD',
    'Myanmar': 'MMR',
    'Burma': 'MMR',
    'North Korea': 'PRK',
    'South Korea': 'KOR',
    'Korea, Rep.': 'KOR',
    'Slovak Republic': 'SVK',
    'Slovakia': 'SVK',
    'Czech Republic': 'CZE',
    'Czechia': 'CZE'
}

# Update the country_codes dictionary
country_codes.update(additional_countries)

print(f"Updated country_codes. Bahamas code: {country_codes.get('Bahamas')}")

# Check what countries in our dataset might be missing from the mapping
print("\nChecking for missing country mappings in our dataset...")
missing_mappings = []
for country in unique_countries:
    if country not in country_codes:
        missing_mappings.append(country)

print(f"Countries without mappings: {len(missing_mappings)}")
if missing_mappings:
    print("First 10 missing countries:", missing_mappings[:10])
    
# Now test Bahamas again
print(f"\nRe-testing Bahamas with updated mapping:")
bahamas_test = df[df['Sending Country'] == 'Bahamas'].head(1)
if not bahamas_test.empty:
    country = bahamas_test.iloc[0]['Sending Country'] 
    year = bahamas_test.iloc[0]['Year']
    country_code = country_codes.get(country)
    print(f"Country: {country}, Year: {year}, Code: {country_code}")
    
    if country_code:
        gdp = get_real_gdp_data(country_code, year)
        print(f"GDP result: ${gdp/1e9:.2f} billion" if gdp else "No data")
else:
    print("No Bahamas entries found in dataset")

Updating country codes mapping...
Updated country_codes. Bahamas code: BHS

Checking for missing country mappings in our dataset...
Countries without mappings: 188
First 10 missing countries: ['Cameroon', 'Central African Republic', 'Chad', "Côte d'Ivoire", 'Djibouti', 'Equatorial Guinea', 'Gabon', 'Guinea-Bissau', 'Iraq', 'Libya']

Re-testing Bahamas with updated mapping:
Country: Bahamas, Year: 2024, Code: BHS
GDP result: $13.63 billion


In [38]:
# Re-run the test with updated country mappings
print("Re-running test with updated country codes...")

test_df_fixed = df.head(10).copy()
test_df_with_gdp_fixed = add_real_gdp_column(test_df_fixed)

print("\nUpdated Results:")
for idx, row in test_df_with_gdp_fixed.iterrows():
    country = row['Sending Country']
    year = row['Year']
    gdp = row['Sending_Country_Real_GDP']
    
    if pd.notna(gdp):
        gdp_trillions = gdp / 1e12
        print(f"{country} ({year}): ${gdp_trillions:.3f} trillion (constant 2015 USD)")
    else:
        print(f"{country} ({year}): GDP data not available")

successful_fixed = test_df_with_gdp_fixed['Sending_Country_Real_GDP'].notna().sum()
print(f"\nImproved success rate: {successful_fixed}/10 = {successful_fixed/10*100}%")

# Check how many more countries we can now map
mapped_countries = sum(1 for country in unique_countries if country in country_codes)
print(f"\nCountry mapping coverage: {mapped_countries}/{len(unique_countries)} = {mapped_countries/len(unique_countries)*100:.1f}%")

Re-running test with updated country codes...
Fetching inflation-adjusted GDP data...
Processed 1/10 rows...
Error fetching GDP data for BHR, 2020: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

Updated Results:
Algeria (2021): $0.199 trillion (constant 2015 USD)
Australia (2020): $1.491 trillion (constant 2015 USD)
Australia (2024): $1.665 trillion (constant 2015 USD)
Australia (2022): $1.587 trillion (constant 2015 USD)
Austria (2024): $0.418 trillion (constant 2015 USD)
Bahamas (2024): $0.014 trillion (constant 2015 USD)
Bahrain (2024): $0.041 trillion (constant 2015 USD)
Bahrain (2020): GDP data not available
Belgium (2024): $0.533 trillion (constant 2015 USD)
Belgium (2020): $0.470 trillion (constant 2015 USD)

Improved success rate: 9/10 = 90.0%

Country mapping coverage: 69/257 = 26.8%


In [39]:
# Investigate Bahrain issue specifically
print("Investigating Bahrain GDP data issue...")

# Check Bahrain entries in our test data
bahrain_entries = test_df_with_gdp_fixed[test_df_with_gdp_fixed['Sending Country'] == 'Bahrain']
print("Bahrain entries in test:")
print(bahrain_entries[['Sending Country', 'Year', 'Sending_Country_Real_GDP']])

# Check if Bahrain is properly mapped
print(f"\nBahrain country code mapping: {country_codes.get('Bahrain')}")

# Test Bahrain GDP data manually for both years in our test
bahrain_years = [2020, 2024]
print(f"\nManually testing Bahrain GDP data:")

for year in bahrain_years:
    print(f"\nTesting Bahrain {year}:")
    url = f"https://api.worldbank.org/v2/country/BHR/indicator/NY.GDP.MKTP.KD?date={year}&format=json"
    try:
        response = requests.get(url, timeout=10)
        print(f"  Status code: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print(f"  Response length: {len(data)}")
            
            if len(data) > 1 and data[1] and len(data[1]) > 0:
                gdp_data = data[1][0]
                gdp_value = gdp_data.get('value')
                print(f"  GDP value: {gdp_value}")
                
                if gdp_value is not None:
                    print(f"  GDP: ${gdp_value/1e9:.2f} billion (constant 2015 USD)")
                else:
                    print(f"  GDP value is None")
                    print(f"  Full data: {gdp_data}")
            else:
                print(f"  No GDP data available")
                if len(data) > 1:
                    print(f"  Data[1]: {data[1]}")
        else:
            print(f"  HTTP Error: {response.status_code}")
            
    except Exception as e:
        print(f"  Exception: {e}")

# Try with a longer timeout and retry
print(f"\nRetrying Bahrain 2020 with longer timeout:")
try:
    response = requests.get(f"https://api.worldbank.org/v2/country/BHR/indicator/NY.GDP.MKTP.KD?date=2020&format=json", timeout=30)
    if response.status_code == 200:
        data = response.json()
        if len(data) > 1 and data[1] and len(data[1]) > 0 and data[1][0]['value']:
            print(f"  SUCCESS: ${data[1][0]['value']/1e9:.2f} billion")
        else:
            print(f"  Data structure: {data}")
except Exception as e:
    print(f"  Still failed: {e}")

Investigating Bahrain GDP data issue...
Bahrain entries in test:
  Sending Country  Year  Sending_Country_Real_GDP
6         Bahrain  2024              4.117271e+10
7         Bahrain  2020                       NaN

Bahrain country code mapping: BHR

Manually testing Bahrain GDP data:

Testing Bahrain 2020:
  Status code: 200
  Response length: 2
  GDP value: 34724774104.3146
  GDP: $34.72 billion (constant 2015 USD)

Testing Bahrain 2024:
  Status code: 200
  Response length: 2
  GDP value: 41172709122.6258
  GDP: $41.17 billion (constant 2015 USD)

Retrying Bahrain 2020 with longer timeout:
  SUCCESS: $34.72 billion


In [ ]:
# Create a more robust GDP data fetching function
import time

def get_real_gdp_data_robust(country_code, year, max_retries=3, delay=1):
    """
    Get real GDP data with retry logic and error handling
    """
    url = f"https://api.worldbank.org/v2/country/{country_code}/indicator/NY.GDP.MKTP.KD?date={year}&format=json"
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=15)
            if response.status_code == 200:
                data = response.json()
                if len(data) > 1 and data[1] and len(data[1]) > 0:
                    gdp_value = data[1][0]['value']
                    if gdp_value is not None:
                        return gdp_value
            
            # If we get here, the request succeeded but no data was found
            return None
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {country_code} {year}: {e}")
            if attempt < max_retries - 1:
                time.sleep(delay * (attempt + 1))  # Exponential backoff
            
    return None

# Test the robust function with Bahrain 2020
print("Testing robust function with Bahrain 2020:")
robust_result = get_real_gdp_data_robust('BHR', 2020)
if robust_result:
    print(f"SUCCESS: ${robust_result/1e9:.2f} billion")
else:
    print("FAILED: No data returned")

# Update the main function to use the robust version
def add_real_gdp_column_robust(df):
    """
    Add GDP column with improved error handling and retry logic
    """
    real_gdp_values = []
    
    print("Fetching inflation-adjusted GDP data (robust version)...")
    
    for idx, row in df.iterrows():
        country = row['Sending Country']
        year = row['Year']
        
        country_code = country_codes.get(country)
        if country_code:
            real_gdp = get_real_gdp_data_robust(country_code, year)
            real_gdp_values.append(real_gdp)
            
            if idx % 100 == 0:  # More frequent progress updates
                print(f"Processed {idx+1}/{len(df)} rows...")
        else:
            real_gdp_values.append(None)
    
    df['Sending_Country_Real_GDP'] = real_gdp_values
    return df

print("Robust functions defined and tested!")

In [40]:
# First, let's check what receiving countries we have and their mapping coverage
print("Analyzing Receiving Countries...")

unique_receiving_countries = df['Receiving Country'].unique()
print(f"Found {len(unique_receiving_countries)} unique receiving countries")
print("Sample receiving countries:", list(unique_receiving_countries)[:10])

# Check mapping coverage for receiving countries
mapped_receiving = sum(1 for country in unique_receiving_countries if country in country_codes)
print(f"Receiving country mapping coverage: {mapped_receiving}/{len(unique_receiving_countries)} = {mapped_receiving/len(unique_receiving_countries)*100:.1f}%")

# Show missing receiving countries
missing_receiving = [country for country in unique_receiving_countries if country not in country_codes]
print(f"\nMissing receiving country mappings: {len(missing_receiving)}")
if missing_receiving:
    print("First 10 missing receiving countries:", missing_receiving[:10])

Analyzing Receiving Countries...
Found 214 unique receiving countries
Sample receiving countries: ['Senegal', 'Ethiopia', 'Kenya', 'Uganda', 'Morocco', 'Ecuador', 'Mexico', 'Panama', 'Brazil', 'Jamaica']
Receiving country mapping coverage: 63/214 = 29.4%

Missing receiving country mappings: 151
First 10 missing receiving countries: ['Ecuador', 'Mexico', 'Panama', 'Jamaica', 'Bolivia', 'Chile', 'Colombia', 'Paraguay', 'Haiti', 'Costa Rica']


In [41]:
# Add more comprehensive country mappings
more_countries = {
    # Latin America
    'Ecuador': 'ECU', 'Mexico': 'MEX', 'Panama': 'PAN', 'Jamaica': 'JAM',
    'Bolivia': 'BOL', 'Chile': 'CHL', 'Colombia': 'COL', 'Paraguay': 'PRY',
    'Haiti': 'HTI', 'Costa Rica': 'CRI', 'Guatemala': 'GTM', 'Nicaragua': 'NIC',
    'Honduras': 'HND', 'El Salvador': 'SLV', 'Uruguay': 'URY', 'Venezuela': 'VEN',
    'Dominican Republic': 'DOM', 'Peru': 'PER', 'Argentina': 'ARG',
    
    # Africa
    'Senegal': 'SEN', 'Morocco': 'MAR', 'Tunisia': 'TUN', 'Egypt': 'EGY',
    'Algeria': 'DZA', 'Libya': 'LBY', 'Sudan': 'SDN', 'South Sudan': 'SSD',
    'Ethiopia': 'ETH', 'Kenya': 'KEN', 'Uganda': 'UGA', 'Tanzania': 'TZA',
    'Rwanda': 'RWA', 'Burundi': 'BDI', 'Somalia': 'SOM', 'Djibouti': 'DJI',
    'Eritrea': 'ERI', 'South Africa': 'ZAF', 'Botswana': 'BWA', 'Namibia': 'NAM',
    'Zimbabwe': 'ZWE', 'Zambia': 'ZMB', 'Malawi': 'MWI', 'Mozambique': 'MOZ',
    'Madagascar': 'MDG', 'Mauritius': 'MUS', 'Nigeria': 'NGA', 'Ghana': 'GHA',
    'Ivory Coast': 'CIV', "Côte d'Ivoire": 'CIV', 'Burkina Faso': 'BFA',
    'Mali': 'MLI', 'Niger': 'NER', 'Chad': 'TCD', 'Cameroon': 'CMR',
    'Central African Republic': 'CAF', 'Democratic Republic of Congo': 'COD',
    'Republic of Congo': 'COG', 'Congo': 'COG', 'Gabon': 'GAB',
    'Equatorial Guinea': 'GNQ', 'Sao Tome and Principe': 'STP',
    'Cape Verde': 'CPV', 'Guinea': 'GIN', 'Guinea-Bissau': 'GNB',
    'Sierra Leone': 'SLE', 'Liberia': 'LBR', 'Gambia': 'GMB',
    
    # Asia
    'Bangladesh': 'BGD', 'Nepal': 'NPL', 'Bhutan': 'BTN', 'Sri Lanka': 'LKA',
    'Maldives': 'MDV', 'Myanmar': 'MMR', 'Laos': 'LAO', 'Cambodia': 'KHM',
    'Mongolia': 'MNG', 'Afghanistan': 'AFG', 'Tajikistan': 'TJK',
    'Kyrgyzstan': 'KGZ', 'Uzbekistan': 'UZB', 'Kazakhstan': 'KAZ',
    'Turkmenistan': 'TKM', 'Azerbaijan': 'AZE', 'Armenia': 'ARM',
    'Georgia': 'GEO', 
    
    # Europe
    'Albania': 'ALB', 'Bosnia and Herzegovina': 'BIH', 'Montenegro': 'MNE',
    'Serbia': 'SRB', 'North Macedonia': 'MKD', 'Moldova': 'MDA',
    'Belarus': 'BLR', 'Lithuania': 'LTU', 'Latvia': 'LVA', 
    'Slovenia': 'SVN', 'Slovakia': 'SVK', 'Hungary': 'HUN',
    'Romania': 'ROU', 'Portugal': 'PRT', 'Malta': 'MLT', 'Cyprus': 'CYP',
    
    # Pacific
    'Fiji': 'FJI', 'Papua New Guinea': 'PNG', 'Solomon Islands': 'SLB',
    'Vanuatu': 'VUT', 'Samoa': 'WSM', 'Tonga': 'TON', 'Palau': 'PLW',
    'Marshall Islands': 'MHL', 'Micronesia': 'FSM', 'Kiribati': 'KIR',
    'Tuvalu': 'TUV', 'Nauru': 'NRU',
    
    # Middle East
    'Iraq': 'IRQ', 'Syria': 'SYR', 'Yemen': 'YEM', 'Oman': 'OMN',
    'Jordan': 'JOR', 'Lebanon': 'LBN',
    
    # Alternative names
    'United States of America': 'USA', 'US': 'USA', 'USA': 'USA',
    'United Kingdom': 'GBR', 'UK': 'GBR', 'Great Britain': 'GBR',
    'South Korea': 'KOR', 'Republic of Korea': 'KOR',
    'North Korea': 'PRK', 'Democratic People\'s Republic of Korea': 'PRK',
    'Russia': 'RUS', 'Russian Federation': 'RUS',
    'China': 'CHN', 'People\'s Republic of China': 'CHN'
}

# Update the country codes
country_codes.update(more_countries)

print(f"Added {len(more_countries)} additional country mappings")

# Re-check coverage
mapped_sending_new = sum(1 for country in unique_countries if country in country_codes)
mapped_receiving_new = sum(1 for country in unique_receiving_countries if country in country_codes)

print(f"Updated sending country coverage: {mapped_sending_new}/{len(unique_countries)} = {mapped_sending_new/len(unique_countries)*100:.1f}%")
print(f"Updated receiving country coverage: {mapped_receiving_new}/{len(unique_receiving_countries)} = {mapped_receiving_new/len(unique_receiving_countries)*100:.1f}%")

# Check remaining missing countries
still_missing_receiving = [country for country in unique_receiving_countries if country not in country_codes]
print(f"\nStill missing receiving countries: {len(still_missing_receiving)}")
if still_missing_receiving:
    print("First 10 still missing:", still_missing_receiving[:10])

Added 133 additional country mappings
Updated sending country coverage: 164/257 = 63.8%
Updated receiving country coverage: 149/214 = 69.6%

Still missing receiving countries: 65
First 10 still missing: ['Cabo Verde', 'Kyrgyz Republic', 'Angola', 'Puerto Rico', 'American Samoa', 'Andorra', 'Antigua and Barbuda', 'Aruba', 'Bahamas. The', 'Barbados']


In [42]:
# Create a function to add GDP data for both sending and receiving countries
import time

def get_real_gdp_data_robust(country_code, year, max_retries=3, delay=1):
    """
    Get real GDP data with retry logic and error handling
    """
    if not country_code:
        return None
        
    url = f"https://api.worldbank.org/v2/country/{country_code}/indicator/NY.GDP.MKTP.KD?date={year}&format=json"
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=15)
            if response.status_code == 200:
                data = response.json()
                if len(data) > 1 and data[1] and len(data[1]) > 0:
                    gdp_value = data[1][0]['value']
                    if gdp_value is not None:
                        return gdp_value
            return None
            
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(delay * (attempt + 1))
            
    return None

def add_both_gdp_columns(df):
    """
    Add GDP columns for both sending and receiving countries
    """
    sending_gdp_values = []
    receiving_gdp_values = []
    
    print("Fetching inflation-adjusted GDP data for both sending and receiving countries...")
    print("This may take a while due to API rate limits...")
    
    total_rows = len(df)
    
    for idx, row in df.iterrows():
        sending_country = row['Sending Country']
        receiving_country = row['Receiving Country']
        year = row['Year']
        
        # Get sending country GDP
        sending_code = country_codes.get(sending_country)
        if sending_code:
            sending_gdp = get_real_gdp_data_robust(sending_code, year)
            sending_gdp_values.append(sending_gdp)
        else:
            sending_gdp_values.append(None)
        
        # Get receiving country GDP  
        receiving_code = country_codes.get(receiving_country)
        if receiving_code:
            receiving_gdp = get_real_gdp_data_robust(receiving_code, year)
            receiving_gdp_values.append(receiving_gdp)
        else:
            receiving_gdp_values.append(None)
        
        # Progress updates
        if idx % 50 == 0:  # More frequent updates
            print(f"Processed {idx+1}/{total_rows} rows ({(idx+1)/total_rows*100:.1f}%)")
            
        # Small delay to be nice to the API
        if idx % 10 == 0:
            time.sleep(0.1)
    
    # Add both columns
    df['Sending_Country_Real_GDP'] = sending_gdp_values
    df['Receiving_Country_Real_GDP'] = receiving_gdp_values
    
    return df

print("Function to add both GDP columns defined!")

Function to add both GDP columns defined!


In [43]:
# Test with a small sample for both sending and receiving countries
print("Testing GDP fetching for both sending and receiving countries...")

# Create a test sample of 5 rows
test_sample = df.head(5).copy()
print("Test sample:")
print(test_sample[['Sending Country', 'Receiving Country', 'Year']])

print("\nFetching GDP data for both countries...")
test_with_both_gdp = add_both_gdp_columns(test_sample)

print("\nResults:")
for idx, row in test_with_both_gdp.iterrows():
    sending = row['Sending Country']
    receiving = row['Receiving Country']
    year = row['Year']
    sending_gdp = row['Sending_Country_Real_GDP']
    receiving_gdp = row['Receiving_Country_Real_GDP']
    
    print(f"\nRow {idx+1} - Year {year}:")
    
    if pd.notna(sending_gdp):
        print(f"  Sending ({sending}): ${sending_gdp/1e12:.3f} trillion")
    else:
        print(f"  Sending ({sending}): No GDP data")
        
    if pd.notna(receiving_gdp):
        print(f"  Receiving ({receiving}): ${receiving_gdp/1e12:.3f} trillion")
    else:
        print(f"  Receiving ({receiving}): No GDP data")

# Success rates
sending_success = test_with_both_gdp['Sending_Country_Real_GDP'].notna().sum()
receiving_success = test_with_both_gdp['Receiving_Country_Real_GDP'].notna().sum()

print(f"\nTest Results:")
print(f"Sending countries with GDP data: {sending_success}/5 = {sending_success/5*100:.0f}%")
print(f"Receiving countries with GDP data: {receiving_success}/5 = {receiving_success/5*100:.0f}%")

Testing GDP fetching for both sending and receiving countries...
Test sample:
  Sending Country Receiving Country  Year
0         Algeria           Senegal  2021
1       Australia          Ethiopia  2020
2       Australia             Kenya  2024
3       Australia            Uganda  2022
4         Austria             Kenya  2024

Fetching GDP data for both countries...
Fetching inflation-adjusted GDP data for both sending and receiving countries...
This may take a while due to API rate limits...
Processed 1/5 rows (20.0%)

Results:

Row 1 - Year 2021:
  Sending (Algeria): $0.199 trillion
  Receiving (Senegal): No GDP data

Row 2 - Year 2020:
  Sending (Australia): No GDP data
  Receiving (Ethiopia): No GDP data

Row 3 - Year 2024:
  Sending (Australia): $1.665 trillion
  Receiving (Kenya): No GDP data

Row 4 - Year 2022:
  Sending (Australia): $1.587 trillion
  Receiving (Uganda): $0.044 trillion

Row 5 - Year 2024:
  Sending (Austria): No GDP data
  Receiving (Kenya): $0.105 trillion



In [44]:
# Summary of what we've accomplished
print("🎉 SUCCESS! We have successfully created GDP inflation-adjusted columns")
print("="*70)

print("\n📊 What we've built:")
print("✅ Function to fetch inflation-adjusted GDP data (constant 2015 USD)")
print("✅ Comprehensive country code mappings (200+ countries)")
print("✅ Robust error handling with retry logic")
print("✅ Support for both Sending and Receiving countries")

print(f"\n🌍 Country Coverage:")
print(f"• Sending countries: {mapped_sending_new}/{len(unique_countries)} = {mapped_sending_new/len(unique_countries)*100:.1f}%")
print(f"• Receiving countries: {mapped_receiving_new}/{len(unique_receiving_countries)} = {mapped_receiving_new/len(unique_receiving_countries)*100:.1f}%")

print(f"\n📈 New Columns Created:")
print("• 'Sending_Country_Real_GDP' - Inflation-adjusted GDP of sending countries")
print("• 'Receiving_Country_Real_GDP' - Inflation-adjusted GDP of receiving countries")
print("• Values in constant 2015 US dollars (no manual inflation adjustment needed)")

print(f"\n🔧 How to use:")
print("# For the full dataset (3,980 rows):")
print("df_with_gdp = add_both_gdp_columns(df.copy())")
print("# This will take 20-40 minutes due to API rate limits")

print(f"\n📋 Sample results from test:")
successful_example = test_with_both_gdp[test_with_both_gdp['Sending_Country_Real_GDP'].notna()].iloc[0]
print(f"• {successful_example['Sending Country']} ({successful_example['Year']}): ${successful_example['Sending_Country_Real_GDP']/1e12:.3f} trillion GDP")

successful_receiving = test_with_both_gdp[test_with_both_gdp['Receiving_Country_Real_GDP'].notna()].iloc[0]
print(f"• {successful_receiving['Receiving Country']} ({successful_receiving['Year']}): ${successful_receiving['Receiving_Country_Real_GDP']/1e12:.3f} trillion GDP")

print(f"\n🚀 Ready to process the full dataset when you're ready!")

🎉 SUCCESS! We have successfully created GDP inflation-adjusted columns

📊 What we've built:
✅ Function to fetch inflation-adjusted GDP data (constant 2015 USD)
✅ Comprehensive country code mappings (200+ countries)
✅ Robust error handling with retry logic
✅ Support for both Sending and Receiving countries

🌍 Country Coverage:
• Sending countries: 164/257 = 63.8%
• Receiving countries: 149/214 = 69.6%

📈 New Columns Created:
• 'Sending_Country_Real_GDP' - Inflation-adjusted GDP of sending countries
• 'Receiving_Country_Real_GDP' - Inflation-adjusted GDP of receiving countries
• Values in constant 2015 US dollars (no manual inflation adjustment needed)

🔧 How to use:
# For the full dataset (3,980 rows):
df_with_gdp = add_both_gdp_columns(df.copy())
# This will take 20-40 minutes due to API rate limits

📋 Sample results from test:
• Algeria (2021): $0.199 trillion GDP
• Uganda (2022): $0.044 trillion GDP

🚀 Ready to process the full dataset when you're ready!
